# Introduction



---



---



In this notebook, I implement the following regime:



---



1) Setup


---



a) $x_{old}.interpolate(\xi).$

b) $u_{old}.interpolate(x_{old}).$

c) Do a physical solve and a mesh solve at a starting $\alpha_{best}$ to get
things started. (NOT SURE THIS IS NEEDED?)

d) This all gives a starting $\alpha_{best}, x_{best}, u_{best}.$



---




2) $\epsilon$-Continuation


---



  a) $\alpha$_list = $[\alpha_{best}, \alpha_{best}/2, \alpha_{best}/4, \alpha_{best}/8, 2 \alpha_{best}, 4 \alpha_{best}, 8 \alpha_{best} ]$

  b) For $\alpha$ in $\alpha$_list:

  - MP-Iteration at current $\alpha$. Use $x_{best}$ and $u_{best}$ as the starting initial guesses
  - Compute an over resolved $u_{fine}$ as our reference solution.
  - Compute the L2 error between the $u$ that results from the MP-Iteration and $u_{fine}$

c) Pick a new $\alpha_{best} = \min(\text{L2 Errors}).$ Choose the $x$ and $u$ computed at that $\alpha$ to be $x_{best}$ and $u_{best}.$ If $\alpha_{best}$ is the smallest or largest values in $\alpha$_list, extend the list in that direction and return to step (b). Only continue solutions between alpha values if the error of the solution at the current alpha is less than 1e-5.




July 2025

# Results and Comments


---



---



- We sort-of abandoned this notebook.

- These results, amoung many others, taught us that we needed to make better informed choices for alpha that depended on epsilon.

- We werent going to pick a magic constant or do the sketchy HR alpha solve. Rather, we are going to use what we know about the width of the layers (for the linear problem) to pick an alpha based on epsilon. After further testing we end up choosing

$$\alpha = \text{constant} \times \epsilon^{-3/4}$$


# Imports

In [ ]:
# install firedrake

# hide output
%%capture

# try:
#     import firedrake
# except ImportError:
#     !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
#     import firedrake


try:
    from firedrake import *
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    from firedrake import *

In [ ]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window

%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [ ]:
# import firedrake tools

from firedrake import *
import numpy as np
import matplotlib.pyplot as plt # firedrake makes use of matplotlib tools
from firedrake.pyplot import tripcolor, tricontour, triplot #firedrake plotting
from IPython.display import display
from mpl_toolkits import mplot3d



# SetUp

In [ ]:
# SETUP
#################################################

# Grid size and epsilon
N = 50
epsilon = 0.1
eps_final = 1e-3
eps_change = 3/4
w = 1/2
mesh_tol = 1e-8
end_MP = False
MP_Iters = 100
max_iter_counter = 0
MP_fails = []
bad_alphas_picked = []

# Set our parameters for the hessian mesh density
p = Constant(2)
q = Constant(2)
m = Constant(0)
exp_M = (2*q) / (1 + q*(2-m))
exp_rho = exp_M/2
RHS = float(2.0)


# Starting alpha
alpha_hat_start = 50



# Function spaces and mesh
uni_mesh = UnitIntervalMesh(N)
uni_coord = SpatialCoordinate(uni_mesh)[0]
V = FunctionSpace(uni_mesh, "CG", 4)
W = FunctionSpace(uni_mesh, "CG", 1)

# For the reference physical solution
V_fine = FunctionSpace(uni_mesh, "CG", 8)


# Solver paramaters
newton_params = {
        'snes_type': 'newtonls',
        # 'snes_monitor': None,
        # "snes_converged_reason": None,
        'ksp_type': 'preonly',
        'snes_linesearch_type': 'l2',
        # 'snes_linesearch_monitor': None,
        # 'snes_linesearch_damping': 0.5,
        'pc_type' : 'lu',
        'snes_rtol': 1e-8,
        'snes_stol': 1e-8,
        'snes_max_it': 100,
}

# The Test

In [ ]:
# Setup for Epsilon Continuation

x_old = Function(W).interpolate(uni_coord)
perm_x = np.argsort(x_old.dat.data[:])
u_old = Function(V).interpolate(x_old+3/2)



# STARTING PHYSICAL AND MESH SOLVES AT BEST ALPHA
###############################################################
best_alpha = alpha_hat_start

# Set u and v
u = Function(V)
v = TestFunction(V)
# Jacobian
Jx = x_old.dx(0)
# Transformed derivatives
u_deriv = (1/Jx) * u.dx(0)
v_deriv = (1/Jx) * v.dx(0)
# Weak form
F1 = ( (epsilon**2) * u_deriv * v_deriv) * Jx * dx
F2 = (u * (u-1) * (u - x_old - 3/2) * v) * Jx * dx
F = F1 + F2
# BC's
bcs = [DirichletBC(V, 3/2, 1), DirichletBC(V, 1/2, 2)]
# Initial guess
u.dat.data[:] = u_old.dat.data[:]
# solve physical problem
solve( F == 0, u, bcs = bcs, solver_parameters = newton_params)


# x jacobian
jacobx = x_old.dx(0) #dx/dxi
# compute the derivatives of our u
u_pr = u_old.dx(0)
u_prpr = u_pr.dx(0)
x_prpr = jacobx.dx(0)
du_dx = (1/jacobx) * u_pr
du2_dx2 = ( u_prpr -  du_dx * x_prpr) * (( 1/jacobx )**2)
# Mesh density function
alpha_term1 = float(best_alpha * sqrt(assemble(u_old*u_old* jacobx * dx)))
M = ( 1 + (1/alpha_term1) * abs(du2_dx2) )**exp_M

# set solution and test function
new_x = Function(W)
vm = TestFunction(W)
# jacobian
Jm = new_x.dx(0) #dx/dxi
# v'
v_pr_term = (vm).dx(0)
# the lhs of the eqn
F = (M * Jm * v_pr_term) * dx
# bc's
bc0 = DirichletBC(W, Constant(0), 1)
bc1 = DirichletBC(W, Constant(1), 2)
bcs = [bc0, bc1]
# initial guess (uniform)
new_x.dat.data[:] = x_old.dat.data[:]
# solve
solve( F == 0, new_x, bcs = bcs, solver_parameters=newton_params)


# Update
x_guess = Function(W)
u_guess = Function(V)
x_guess.dat.data[:] = new_x.dat.data[:]
u_guess.dat.data[:] = u.dat.data[:]


# # Now save our 'best solutions'
x_best = Function(W)
u_best = Function(V)
x_best.dat.data[:] = new_x.dat.data[:]
u_best.dat.data[:] = u.dat.data[:]




while epsilon >= eps_final:



    print()
    print('-_'*100)
    print()
    print("epsilon = ", epsilon)
    print()
    print('_-'*100)
    print()
    print()


    # Update our guesses - CONTINUATION MOMENT!
    x_guess.dat.data[:] = x_best.dat.data[:]
    u_guess.dat.data[:] = u_best.dat.data[:]

    print('Continuation check')
    print('norm(x guess) = ', norm(x_guess))
    print('norm(u guess) = ', norm(u_guess))
    print()
    print()


    # Try out versions of our current best alpha
    alpha_list = [ best_alpha * (eps_change) ** p for p in [-2, -1, 0, 1, 2]]


    # Set min and max alpha
    min_alpha = min(alpha_list)
    max_alpha = max(alpha_list)

    # Save errors and solutions for each alpha
    alpha_errors = []
    u_x_tracker = []

    # Check if we choose a bad alpha (alpha for which the MP Iteration doesnt comverge)
    alpha_bad = []



    for alpha_val in alpha_list:



        # Update alpha_hat
        alpha_hat = Constant(alpha_val)

        print()
        print('_'*100)
        print()
        print("alpha_hat = ", alpha_val)
        print()
        print('_'*100)
        print()

        # # Our initial guesses
        u_old.dat.data[:] = u_guess.dat.data[:]
        x_old.dat.data[:] = x_guess.dat.data[:]

        # continuation check
        print('Continuation check')
        print('norm(x) = ', norm(x_old))
        print('norm(u) = ', norm(u_old))
        print()
        print()


        # If a solver fails, this will move us to next alpha
        forced_break = False


        # MP ITERATION
        ########################################################################

        mesh_norm_tracker = []

        for i in range(MP_Iters):


            # SET OUR M
            #############################################

            # x jacobian
            jacobx = x_old.dx(0) #dx/dxi

            # compute the derivatives of our u
            u_pr = u_old.dx(0)
            u_prpr = u_pr.dx(0)
            x_prpr = jacobx.dx(0)
            du_dx = (1/jacobx) * u_pr
            du2_dx2 = ( u_prpr -  du_dx * x_prpr) * (( 1/jacobx )**2)


            # Mesh density function
            alpha_term1 = float(alpha_hat * sqrt(assemble(u_old*u_old* jacobx * dx)))
            M = ( 1 + (1/alpha_term1) * abs(du2_dx2) )**exp_M


            # plt.plot(x_old.dat.data[perm_x], Function(W).interpolate(du2_dx2).dat.data[perm_x], label = "du2_dx2")
            # plt.plot(x_old.dat.data[perm_x], [0 for i in range(len(x_old.dat.data[perm_x]))], marker = "|")
            # plt.title("u'' plot")
            # plt.xlabel('x')
            # plt.ylabel('du2_dx2')
            # plt.legend()
            # plt.show()
            # plt.close()
            # print()
            # print()



            # MESH SOLVE WEAK FORM
            # ######################################


            # set solution and test function
            new_x = Function(W)
            vm = TestFunction(W)

            # jacobian
            Jm = new_x.dx(0) #dx/dxi

            # v'
            v_pr_term = (vm).dx(0)

            # the lhs of the eqn
            F = (M * Jm * v_pr_term) * dx

            # bc's
            bc0 = DirichletBC(W, Constant(0), 1)
            bc1 = DirichletBC(W, Constant(1), 2)
            bcs = [bc0, bc1]


            # initial guess (uniform)
            new_x.dat.data[:] = x_old.dat.data[:]


            # MESH SOLVE
            #######################################

            try:
              solve( F == 0, new_x, bcs = bcs, solver_parameters=newton_params)
            except Exception as e:
              print('MESH SOLVE FAILED BECAUSE ', e)
              print()
              print()
              forced_break = True
              break

            # compute the l2 norm of xnew-x_old
            l2_check = errornorm(new_x, x_old)

            # Save the norm to look at later
            mesh_norm_tracker.append(l2_check)


            # Do we have mesh convergence?
            if l2_check <= mesh_tol:
              end_MP = True


            # update our x_old and u_interp - MP ITERATION CONTINUATION
            x_old.interpolate( (1-w) * x_old + w * new_x )



            # PHYSICAL SOLVE ON THE NEW MESH
            #######################################

            u = Function(V)
            v = TestFunction(V)
            J_xnew = new_x.dx(0)
            u_deriv = (1/J_xnew) * u.dx(0)
            v_deriv = (1/J_xnew) * v.dx(0)


            # problem setup
            F1 = ( (epsilon**2) * u_deriv * v_deriv) * J_xnew * dx
            F2 = (u * (u-1) * (u - new_x - 3/2) * v) * J_xnew * dx
            F = F1 + F2

            # BC's
            bcs = [DirichletBC(V, 3/2, 1), DirichletBC(V, 1/2, 2)]

            # Initial guess
            u.dat.data[:] = u_old.dat.data[:]

            # solve physical problem
            try:
              solve( F == 0, u, bcs = bcs, solver_parameters = newton_params)
            except Exception as e:
              print('PHYSICAL SOLVE FAILED BECAUSE ', e)
              print()
              print()
              forced_break = True
              break

            # update u_old - MP ITERATION CONTINUATION
            u_old.dat.data[:] = u.dat.data[:]



            # CHECK MESH CONVERGENCE
            # ######################################

            if end_MP:

              print()
              print('We performed', i+1, 'MP Iterations for epsilon = ', epsilon)
              print()

              break


        # DID WE CONVERGE?
        #########################################


        if forced_break: # This current alpha value failed!

          alpha_errors.append(np.inf)
          u_x_tracker.append([None, None])
          continue

        if not(end_MP):

          print()
          print('We reached MAX MP Iterations!')
          print()
          print('The MP-Iteration mesh norms are :', mesh_norm_tracker)
          print()
          max_iter_counter += 1
          MP_fails.append([epsilon, alpha_val, mesh_norm_tracker])
          alpha_bad.append(alpha_val)


        # RESET MP ITER STOPPING
        ##########################################

        end_MP = False


        # COMPUTE ERROR AT THIS ALPHA CHOICE
        ###############################################################


        # Over Resolved Physical Solve

        # Set u and v
        u_fine = Function(V_fine)
        v_fine = TestFunction(V_fine)
        # Jacobian
        J_fine = x_old.dx(0)
        # Transformed derivatives
        u_pr_fine = (1/J_fine) * u_fine.dx(0)
        v_pr_fine = (1/J_fine) * v_fine.dx(0)
        # Weak form
        F1 = ( (epsilon**2) * u_pr_fine * v_pr_fine) * J_fine * dx
        F2 = (u_fine * (u_fine-1) * (u_fine - x_old - 3/2) * v_fine) * J_fine * dx
        F = F1 + F2
        # BC's
        bcs = [DirichletBC(V_fine, 3/2, 1), DirichletBC(V_fine, 1/2, 2)]
        # Initial guess
        u_fine.interpolate(u_old)
        # solve physical problem
        solve( F == 0, u_fine, bcs = bcs, solver_parameters = newton_params)

        # Plot Comparison
        plt.plot(x_old.dat.data[perm_x], Function(W).interpolate(u_old).dat.data[perm_x], label = "Computed u")
        plt.plot(x_old.dat.data[perm_x], Function(W).interpolate(u_fine).dat.data[perm_x], label = "Over Resolved u")
        plt.plot(x_old.dat.data[perm_x], [0 for i in range(len(x_old.dat.data[perm_x]))], marker = "|")
        plt.title('Comparing Computed u to Over Resolved u at Alpha = '+str(alpha_val))
        plt.xlabel('x')
        plt.ylabel('u')
        plt.legend()
        plt.show()
        plt.close()
        print()
        print()


        # Compute L2 Error
        error_val = (u_old - u_fine)
        L2_error = norm(error_val * (sqrt(abs(x_old.dx(0)))))
        print("L2 error is", L2_error)
        print()
        print()

        alpha_errors.append(L2_error)


        # Save u and x
        u_save = Function(V)
        x_save = Function(W)
        u_save.dat.data[:] = u_old.dat.data[:]
        x_save.dat.data[:] = x_old.dat.data[:]
        u_x_tracker.append([u_save, x_save])





        # Check if we should keep the u and x as initial guesses or not

        if L2_error < (1e-5):

          print('Since L2 Error are small enough, we will update our guesses!')

          # update u guess and x guess - ALPHA CONTINUATION STEP
          u_guess.dat.data[:] = u_old.dat.data[:]
          x_guess.dat.data[:] = x_old.dat.data[:]

          print('Continuation check')
          print('norm(x) = ', norm(x_old))
          print('norm(u) = ', norm(u_old))
          print()



        # Check if we need to extend the alpha list

        if alpha_val == alpha_list[-1]:

            # Check the current best alpha!
            L2_pick = min(alpha_errors)
            L2_index = alpha_errors.index(L2_pick)
            alpha_pick = alpha_list[L2_index]


            # Do we need to extend the alpha list?

            if alpha_pick == max_alpha:

              print()
              print("CURRENT ALPHA BEST IS ", alpha_pick)
              print()
              print('ADDING BIGGER VALUES TO ALPHA LIST')

              alpha_list.extend([alpha_pick*(eps_change)**(-1), alpha_pick*(eps_change)**(-2), alpha_pick*(eps_change)**(-3)])

              print()
              print("CURRENT ALPHA BEST IS ", alpha_pick)
              print()
              print('The new alpha list is', alpha_list)
              print()

            elif alpha_pick == min_alpha:


              print()
              print('ADDING SMALLER VALUES TO ALPHA LIST')

              alpha_list.extend([alpha_pick*(eps_change)**(1), alpha_pick*(eps_change)**(2), alpha_pick*(eps_change)**(3)])

              print('The new alpha list is', alpha_list)
              print()

            # Update min and max alpha
            min_alpha = min(alpha_list)
            max_alpha = max(alpha_list)


        # continuation check
        print('Continuation check')
        print('norm(x guess) = ', norm(x_guess))
        print('norm(u guess) = ', norm(u_guess))
        print()
        print()





    # Pick a New Best Alpha and Save The Results
    best_L2 = min(alpha_errors)
    best_index = alpha_errors.index(best_L2)
    best_alpha = alpha_list[best_index]
    u_best.dat.data[:] = u_x_tracker[best_index][0].dat.data[:]
    x_best.dat.data[:] = u_x_tracker[best_index][1].dat.data[:]

    print()
    print('_'*100)
    print()
    print('Results')
    print()
    print('_'*100)
    print()
    print("New Best Alpha is", best_alpha)
    print()
    print()

    plt.plot(x_best.dat.data[perm_x], Function(W).interpolate(u_best).dat.data[perm_x], label = "best u")
    plt.plot(x_best.dat.data[perm_x], [0 for i in range(len(x_old.dat.data[perm_x]))], marker = "|")
    plt.title('Best x and u for best_alpha = '+str(best_alpha))
    plt.xlabel('x')
    plt.ylabel('u')
    plt.legend()
    plt.show()
    plt.close()

    if best_alpha in alpha_bad:

      print('We picked a bad alpha!')
      print()
      print()
      bad_alphas_picked.append([epsilon, best_alpha])


    epsilon = eps_change * epsilon


    print('Continuation check')
    print('norm(x best) = ', norm(x_best))
    print('norm(u best) = ', norm(u_best))
    print()
    print()






Output hidden; open in https://colab.research.google.com to view.

# Exploring the Failed MP-Iterations

In [ ]:
print('Number of MP-Iter fails:', max_iter_counter)
print()
print('When did the MP-Iter Fail?')
print()

for fail in MP_fails:

  print('epsilon = ', fail[0], end = "   ")
  print('alpha = ', fail[1], end = "   ")
  print('Last 5 L2 mesh checks = ', fail[2][-5:])
  print()

Number of MP-Iter fails: 44

When did the MP-Iter Fail?

epsilon =  0.07500000000000001   alpha =  21.09375   Last 5 L2 mesh checks =  [np.float64(3.9277637958369114e-07), np.float64(3.3585975275842673e-07), np.float64(2.858830711775795e-07), np.float64(2.422118735279442e-07), np.float64(2.0428291413545074e-07)]

epsilon =  0.05625000000000001   alpha =  37.49999999999999   Last 5 L2 mesh checks =  [np.float64(4.2112943769198524e-07), np.float64(4.130371363826433e-07), np.float64(4.0409691740265994e-07), np.float64(3.938481160387109e-07), np.float64(3.8258309231667054e-07)]

epsilon =  0.0421875   alpha =  88.88888888888887   Last 5 L2 mesh checks =  [np.float64(1.5106634983644957e-07), np.float64(1.2985673791007946e-07), np.float64(1.1146873925280343e-07), np.float64(9.551991609810469e-08), np.float64(8.171984192616254e-08)]

epsilon =  0.0421875   alpha =  66.66666666666666   Last 5 L2 mesh checks =  [np.float64(0.004036899165682194), np.float64(0.004036898982341284), np.float64(0.00

In [ ]:
print('Out of the failed alphas, how many were chosen as the best alpha?')
print()

for bad_a in bad_alphas_picked:

  print(bad_a)
  print()

Out of the failed alphas, how many were chosen as the best alpha?

[0.0421875, 88.88888888888887]

[0.01001129150390625, 210.69958847736618]

[0.007508468627929688, 280.93278463648824]

[0.005631351470947266, 280.93278463648824]

[0.004223513603210449, 374.5770461819843]

